In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date 

In [0]:
# Orders Table
orders_data = [
    (1, 101, 1001, date(2025, 10, 1), 250.50),
    (2, 102, 1002, date(2025, 10, 2), 450.75),
    (3, 103, 1003, date(2025, 10, 3), 120.00),
    (4, 104, 1002, date(2025, 10, 4), 300.25),
    (5, 105, 1004, date(2025, 10, 5), 550.00),
    (6, 106, 1005, date(2025, 10, 6), 275.75),
    (7, 107, 1001, date(2025, 10, 7), 220.00),
    (8, 108, 1003, date(2025, 10, 8), 180.50)
]

orders_columns = ["order_id", "customer_id", "product_id", "order_date", "amount"]
orders_df = spark.createDataFrame(orders_data, orders_columns)

# Sales Table
sales_data = [
    (1, 1001, date(2025, 10, 5), 260.00),
    (2, 1002, date(2025, 10, 6), 470.50),
    (3, 1003, date(2025, 10, 7), 130.25),
    (4, 1004, date(2025, 10, 8), 560.00),
    (5, 1005, date(2025, 10, 9), 290.00),
    (6, 1006, date(2025, 10, 10), 330.75),
    (7, 1001, date(2025, 10, 11), 240.25),
    (8, 1002, date(2025, 10, 12), 460.00)
]

sales_columns = ["sale_id", "product_id", "sale_date", "amount"]
sales_df = spark.createDataFrame(sales_data, sales_columns)

# Departments Table
departments_data = [
    (10, "Electronics"),
    (20, "Home Appliances"),
    (30, "Furniture"),
    (40, "Groceries"),
    (50, "Clothing")
]

departments_columns = ["dept_id", "dept_name"]
dept_df = spark.createDataFrame(departments_data, departments_columns)

emp_data = [
    (1, "Adam Scott", 10, 3, 55000, "1998-01-10", "Developer"),
    (2, "Bella Rose", 20, 5, 60000, "2021-03-15", "Analyst"),
    (3, "Charlie King", 10, None, 80000, "2019-05-20", "Manager"),
    (4, "Diana Prince", 20, 5, 62000, "2020-07-25", "Team Lead"),
    (5, "Ethan Hunt", 20, None, 90000, "2018-02-10", "Director"),
    (6, "Fiona Glenanne", 30, 7, 50000, "2022-09-05", "QA Engineer"),
    (7, "George Clooney", 30, None, 85000, "2017-11-30", "Manager"),
    (8, "Hannah Baker", 10, 3, 57000, "2021-12-12", "Developer")
]

# Define column names
emp_columns = ["emp_id", "emp_name", "dept_id", "manager_id", "salary", "hire_date", "job_title"]

# Create DataFrame
emp_df = spark.createDataFrame(emp_data, emp_columns)

# Register as a temporary SQL table
emp_df.createOrReplaceTempView("emp_tbl")

# ----------------------------------------------------------
# 3️⃣ Register as Spark SQL Temporary Tables
# ----------------------------------------------------------

orders_df.createOrReplaceTempView("orders_tbl")
sales_df.createOrReplaceTempView("sales_tbl")
dept_df.createOrReplaceTempView("dept_tbl")


## 1

In [0]:
#Find Second Highest Salary
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,John Doe,10,101,50000,2021-01-15,Developer
2,Jane Smith,20,102,60000,2020-03-10,Manager
3,Alice Johnson,10,101,55000,2022-07-25,Analyst
4,Bob Brown,30,103,45000,2023-02-18,Tester
5,Charlie Davis,20,102,62000,2019-11-05,Team Lead
6,Evelyn White,40,104,70000,2020-12-30,Project Manager
7,Frank Harris,10,101,52000,2021-05-12,Data Engineer
8,Grace Lee,30,103,48000,2022-08-09,QA Engineer
9,Henry Wilson,40,104,75000,2018-09-15,Architect
10,Isabella Green,50,105,68000,2019-04-22,Business Analyst


In [0]:
emp_df.withColumn("rank",dense_rank().over(Window.orderBy(col("salary").desc())))\
    .filter(col("rank")==2)\
    .select("salary")\
    .display()

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1117: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


salary
70000


In [0]:
#Without Window Function
distinct_salaries = emp_df.select("salary").distinct().orderBy(desc("salary"))
second_highest_salary = distinct_salaries.limit(2).collect()[-1][0]
print(second_highest_salary)

70000


In [0]:
from pyspark.sql.window import Window

In [0]:
spark.sql("""
          select max(salary) as max_salary
          from emp_tbl
          where salary<(select max(salary)
          from emp_tbl)
          
          """).display()

max_salary
70000


In [0]:
#select 3rd/nth highest salary
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,John Doe,10,101,50000,2021-01-15,Developer
2,Jane Smith,20,102,60000,2020-03-10,Manager
3,Alice Johnson,10,101,55000,2022-07-25,Analyst
4,Bob Brown,30,103,45000,2023-02-18,Tester
5,Charlie Davis,20,102,62000,2019-11-05,Team Lead
6,Evelyn White,40,104,70000,2020-12-30,Project Manager
7,Frank Harris,10,101,52000,2021-05-12,Data Engineer
8,Grace Lee,30,103,48000,2022-08-09,QA Engineer
9,Henry Wilson,40,104,75000,2018-09-15,Architect
10,Isabella Green,50,105,68000,2019-04-22,Business Analyst


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
emp_df.withColumn("rank",dense_rank().over(Window.orderBy(col("salary").desc())))\
  .filter(col("rank")==3)\
  .select("salary")\
  .display()


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1117: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


salary
68000


In [0]:
spark.sql("""
          with CTE as (
              select *,dense_rank() over(order by salary desc) as rank
              from emp_tbl
          )
          select *
          from CTE
          where rank=3
          """).display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title,rank
10,Isabella Green,50,105,68000,2019-04-22,Business Analyst,3


## 3

In [0]:
#find Duplicte record
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,John Doe,10,101,50000,2021-01-15,Developer
2,Jane Smith,20,102,60000,2020-03-10,Manager
3,Alice Johnson,10,101,55000,2022-07-25,Analyst
4,Bob Brown,30,103,45000,2023-02-18,Tester
5,Charlie Davis,20,102,62000,2019-11-05,Team Lead
6,Evelyn White,40,104,70000,2020-12-30,Project Manager
7,Frank Harris,10,101,52000,2021-05-12,Data Engineer
8,Grace Lee,30,103,48000,2022-08-09,QA Engineer
9,Henry Wilson,40,104,75000,2018-09-15,Architect
10,Isabella Green,50,105,68000,2019-04-22,Business Analyst


In [0]:
emp_df.groupBy("emp_id").agg(count("*").alias("count"))\
  .filter(col("count")>1)\
  .display()

emp_id,count
1,2


In [0]:
spark.sql("""
          select emp_id,count(*)
          from emp_tbl
          group by emp_id
          having count(*)>1
          """).display()

emp_id,count(*)
1,2


## 4
###Delete Duplicate Record

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,John Doe,10,101,50000,2021-01-15,Developer
1,John Doe,10,101,50000,2021-01-15,Developer
1,John Doe,10,101,50000,2021-01-15,Developer
2,Jane Smith,20,102,60000,2020-03-10,Manager
3,Alice Johnson,10,101,55000,2022-07-25,Analyst
4,Bob Brown,30,103,45000,2023-02-18,Tester
5,Charlie Davis,20,102,62000,2019-11-05,Team Lead
6,Evelyn White,40,104,70000,2020-12-30,Project Manager
7,Frank Harris,10,101,52000,2021-05-12,Data Engineer
8,Grace Lee,30,103,48000,2022-08-09,QA Engineer


In [0]:
from pyspark.sql.functions import monotonically_increasing_id
emp_df.withColumn("row_id", monotonically_increasing_id())\
  .withColumn("Rank",dense_rank().over(Window.partitionBy("emp_id").orderBy("row_id")))\
  .filter(col("rank")==1)\
  .display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title,row_id,Rank
1,John Doe,10,101,50000,2021-01-15,Developer,0,1
2,Jane Smith,20,102,60000,2020-03-10,Manager,3,1
3,Alice Johnson,10,101,55000,2022-07-25,Analyst,4,1
4,Bob Brown,30,103,45000,2023-02-18,Tester,5,1
5,Charlie Davis,20,102,62000,2019-11-05,Team Lead,6,1
6,Evelyn White,40,104,70000,2020-12-30,Project Manager,7,1
7,Frank Harris,10,101,52000,2021-05-12,Data Engineer,8,1
8,Grace Lee,30,103,48000,2022-08-09,QA Engineer,9,1
9,Henry Wilson,40,104,75000,2018-09-15,Architect,10,1
10,Isabella Green,50,105,68000,2019-04-22,Business Analyst,11,1


In [0]:
spark.sql("""
          with CTE as (
            select *,rank() over(partition by emp_id order by row_id) as rank
            from emp_tbl
          )

          delete from CTE
          where rank > 1
          
          """).display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6327681490876130>, line 1
----> 1 spark.sql("""
      2           with CTE as (
      3             select *,rank() over(partition by emp_id order by row_id) as rank
      4             from emp_tbl
      5           )
      6 
      7           delete from CTE
      8           where rank > 1
      9           
     10           """).display()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/session.py:872, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    869         _views.append(SubqueryAlias(df._plan, name))
    871 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 872 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    873 if "sql_command_result" in properties:
    874     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /datab

## 5
emp earning more than maneger

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,2022-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
e = emp_df.alias("e")
m = emp_df.alias("m")

# Correct join: employee.manager_id = manager.emp_id
e.join(m, e["manager_id"] == m["emp_id"], "inner") \
 .filter(e["salary"] < m["salary"]) \
 .select(e["emp_name"].alias("emp"), m["emp_name"].alias("manager")) \
 .display()



emp,manager
Hannah Baker,Charlie King
Adam Scott,Charlie King
Diana Prince,Ethan Hunt
Bella Rose,Ethan Hunt
Fiona Glenanne,George Clooney


In [0]:
spark.sql("""
          select e.emp_name,m.emp_name
          from emp_tbl e
          inner join emp_tbl m
          on e.manager_id=m.emp_id
          where e.salary<m.salary
          
          """).display()

emp_name,emp_name
Hannah Baker,Charlie King
Adam Scott,Charlie King
Diana Prince,Ethan Hunt
Bella Rose,Ethan Hunt
Fiona Glenanne,George Clooney


## 6
### EMP joined before their maneger

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,2022-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
e=emp_df.alias("e")
m=emp_df.alias("m")

e.join(m,e.manager_id==m.emp_id,"inner")\
  .filter(e["hire_date"]<m["hire_date"])\
  .select(e["emp_name"],e["hire_date"].alias("emp_hire_date"),m["emp_name"].alias("maneger_name"),m["hire_date"].alias("maneger_hire_date"))\
  .display()

emp_name,emp_hire_date,maneger_name,maneger_hire_date
Adam Scott,1998-01-10,Charlie King,2019-05-20


In [0]:
spark.sql("""
          select e.emp_name,e.hire_date as emp_hire_date,m.emp_name,m.hire_date as maneger_hire_date
          from emp_tbl e
          inner join emp_tbl m
          on e.manager_id=m.emp_id
          where e.hire_date<m.hire_date
          
          
          """).display()

emp_name,emp_hire_date,emp_name,maneger_hire_date
Adam Scott,1998-01-10,Charlie King,2019-05-20


7
### Get rolling average from last 7 days

In [0]:
sales_df.display()

sale_id,product_id,sale_date,amount
1,1001,2025-10-05,260.0
2,1002,2025-10-06,470.5
3,1003,2025-10-07,130.25
4,1004,2025-10-08,560.0
5,1005,2025-10-09,290.0
6,1006,2025-10-10,330.75
7,1001,2025-10-11,240.25
8,1002,2025-10-12,460.0


In [0]:
spark.sql("""
          select *,avg(amount) over(order by sale_date rows between 6 preceding and current row) as roll_avg
          from sales_tbl
          """).display()

sale_id,product_id,sale_date,amount,roll_avg
1,1001,2025-10-05,260.0,260.0
2,1002,2025-10-06,470.5,365.25
3,1003,2025-10-07,130.25,286.9166666666667
4,1004,2025-10-08,560.0,355.1875
5,1005,2025-10-09,290.0,342.15
6,1006,2025-10-10,330.75,340.25
7,1001,2025-10-11,240.25,325.9642857142857
8,1002,2025-10-12,460.0,354.5357142857143


In [0]:
#for each product
spark.sql("""
          select *, avg(amount) over(partition by product_id order by sale_date rows between 6 preceding and current row) as roll_product_avg
          from sales_tbl
          """).display()

sale_id,product_id,sale_date,amount,roll_product_avg
1,1001,2025-10-05,260.0,260.0
7,1001,2025-10-11,240.25,250.125
2,1002,2025-10-06,470.5,470.5
8,1002,2025-10-12,460.0,465.25
3,1003,2025-10-07,130.25,130.25
4,1004,2025-10-08,560.0,560.0
5,1005,2025-10-09,290.0,290.0
6,1006,2025-10-10,330.75,330.75


In [0]:
from pyspark.sql import Window

In [0]:
sales_df.withColumn("roll_avg",avg("amount").over(Window.partitionBy("product_id").orderBy("sale_date").rowsBetween(-6, 0)))\
  .display()


sale_id,product_id,sale_date,amount,roll_avg
1,1001,2025-10-05,260.0,260.0
7,1001,2025-10-11,240.25,250.125
2,1002,2025-10-06,470.5,470.5
8,1002,2025-10-12,460.0,465.25
3,1003,2025-10-07,130.25,130.25
4,1004,2025-10-08,560.0,560.0
5,1005,2025-10-09,290.0,290.0
6,1006,2025-10-10,330.75,330.75


# 8
## Running total of sales

In [0]:
sales_df.display()

sale_id,product_id,sale_date,amount
1,1001,2025-10-05,260.0
2,1002,2025-10-06,470.5
3,1003,2025-10-07,130.25
4,1004,2025-10-08,560.0
5,1005,2025-10-09,290.0
6,1006,2025-10-10,330.75
7,1001,2025-10-11,240.25
8,1002,2025-10-12,460.0


In [0]:
spark.sql("""
          select *,sum(amount) over(order by sale_date) as running_sum
          from sales_tbl
           """).display()

sale_id,product_id,sale_date,amount,running_sum
1,1001,2025-10-05,260.0,260.0
2,1002,2025-10-06,470.5,730.5
3,1003,2025-10-07,130.25,860.75
4,1004,2025-10-08,560.0,1420.75
5,1005,2025-10-09,290.0,1710.75
6,1006,2025-10-10,330.75,2041.5
7,1001,2025-10-11,240.25,2281.75
8,1002,2025-10-12,460.0,2741.75


# 9
## Top 3 salary per department

In [0]:
emp_df.display()

emp_id,emp_name,dept_id,manager_id,salary,hire_date,job_title
1,Adam Scott,10,3,55000,1998-01-10,Developer
2,Bella Rose,20,5,60000,2021-03-15,Analyst
3,Charlie King,10,null,80000,2019-05-20,Manager
4,Diana Prince,20,5,62000,2020-07-25,Team Lead
5,Ethan Hunt,20,null,90000,2018-02-10,Director
6,Fiona Glenanne,30,7,50000,2022-09-05,QA Engineer
7,George Clooney,30,null,85000,2017-11-30,Manager
8,Hannah Baker,10,3,57000,2021-12-12,Developer


In [0]:
spark.sql("""
          with cte as(
              select dept_id,salary,dense_rank() over(partition by dept_id order by salary)as rank
              from emp_tbl
          )

          select * 
          from cte
          where rank<4
          """).display()

dept_id,salary,rank
10,55000,1
10,57000,2
10,80000,3
20,60000,1
20,62000,2
20,90000,3
30,50000,1
30,85000,2


In [0]:
emp_df.withColumn("rank",dense_rank().over(Window.partitionBy("dept_id").orderBy("salary")))\
  .filter(col("rank")<4)\
  .select("dept_id","salary")\
  .display()

dept_id,salary
10,55000
10,57000
10,80000
20,60000
20,62000
20,90000
30,50000
30,85000
